### Permutation mechanism

This notebook implements the mechanism of a "move", and reverse of a move

It also goes through the examples and validates that all solutions are correct.

We also try to optimize a little bit so that each step is fast.

In [1]:
import json
import copy

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# load each puzzle and their moves
puzzle_actions = {}
puzzle_info_arr = pd.read_csv("./puzzles/puzzle_info.csv").to_numpy()
for puzzle_name, moves in puzzle_info_arr:
    puzzle_actions[puzzle_name] = json.loads(moves.replace("'", '"'))

# load all the puzzle's start and end positions
puzzles_arr = pd.read_csv("./puzzles/puzzles.csv").to_numpy()

# load sample solutions
# sample_submission_arr = pd.read_csv("./puzzles/sample_submission.csv").to_numpy()
sample_submission_arr = pd.read_csv("./solutions/greedy_all.csv").to_numpy()

In [4]:
from src.mechanism.permute import permute, permute_with_swap, perm_to_swap, reverse_perm
from src.mechanism.utils import diff_between_states, is_valid_final_state

In [5]:
reduced_actions = {pname: {} for pname in puzzle_actions.keys()}

# add reversed actions all the possible actions
for puzzle_name, actions in puzzle_actions.items():
    reversed_actions = {}
    for action_name, action in actions.items():
        reversed_actions[f"-{action_name}"] = reverse_perm(action)
    puzzle_actions[puzzle_name].update(reversed_actions)

# get reduced actions
for puzzle_name, actions in puzzle_actions.items():
    for action_name, action in actions.items():
        reduced_actions[puzzle_name][action_name] = perm_to_swap(action)

NUM_WRONGS = 0
# play through all the sample submission moves to validate
for puzzle in tqdm(puzzles_arr):
    i, pname, end_state, start_state, num_wild = puzzle
    start_state = start_state.split(";")
    end_state = end_state.split(";")

    # get sample solution
    moves = sample_submission_arr[i][1].split(".")

    # get action space
    actions = puzzle_actions[pname]
    actions = reduced_actions[pname]

    curr_state = start_state
    # play through the submitted actions
    for move_name in moves:
        move = actions[move_name]
        # curr_state = permute(curr_state, move)
        curr_state = permute_with_swap(curr_state, move)

    if is_valid_final_state(curr_state, end_state, num_wild):
        # print(f"validated puzzle {i} on {pname} with {len(moves)} moves")
        pass
    else:
        NUM_WRONGS += 1
        print(f"Puzzle {i} FAILED")

print(f"Failed on {NUM_WRONGS} puzzles")

100%|██████████| 398/398 [00:13<00:00, 29.03it/s]  

Failed on 0 puzzles


In [19]:
moves = sample_submission_arr[0][1].split(".")
moves

['r1', '-f1']

In [29]:
globe = "globe_3/4"
state = list(range(len(puzzle_actions[globe]["r1"])))
move = puzzle_actions[globe]["r0"]

start = state[:]
state = permute(state, move)
c = 1
while start != state:
    state = permute(state, move)
    c += 1

print(c)
print(state)
print(start)

print(",".join([str(n) for n in puzzle_actions["globe_3/4"]["r0"]]))
print(",".join([str(n) for n in puzzle_actions["globe_3/4"]["-r0"]]))

8
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
1,2,3,4,5,6,7,0,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
7,0,1,2,3,4,5,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31


In [31]:
'globe_3/33'.split('/')

['globe_3', '33']